# **Entregable 1 - Extracción de entidades de historias clínicas de cáncer de mama**

**Integrantes:**
- Roberto Ceballos
- Mayra Erazo
- Katheryn Sanchez
- Yeraldin Tafur







Las historias clínicas contienen una gran cantidad de información médica no estructurada escrita en lenguaje natural, como diagnósticos, tratamientos, síntomas y antecedentes del paciente. Esta información es valiosa, pero difícil de analizar automáticamente si no se estructura primero.

La **extracción de entidades nombradas** (**NER**, por sus siglas en inglés) es una técnica del procesamiento de lenguaje natural (PLN) que permite identificar y clasificar automáticamente fragmentos relevantes del texto (entidades), como enfermedades, medicamentos, órganos o procedimientos médicos.

En el contexto del cáncer de mama, esta técnica permite detectar menciones específicas relacionadas con:

- Diagnósticos como “carcinoma ductal infiltrante”

- Biomarcadores como “HER2” o “receptores hormonales”

- Procedimientos como “mastectomía” o “biopsia”

- Lugares anatómicos como “mama izquierda” o “ganglios axilares”

<p align="center">
    <img src="https://i.postimg.cc/0j6FG1Pg/Tarea2-drawio-1.png" width="800">
</p>

El objetivo principal de este script es **automatizar la extracción de entidades biomédicas** desde un conjunto de historias clínicas de pacientes con cáncer de mama, utilizando un modelo NER previamente entrenado.

Este proceso incluye:

1. **Cargar** un modelo NER de Hugging Face.
2. **Leer y procesar** archivos `.txt` que contienen historias clínicas reales.
3. **Detectar** entidades médicas relevantes dentro del texto.
4. **Clasificar** cada entidad según su tipo (diagnóstico, procedimiento, etc.).
5. **Guardar** los resultados estructurados en un archivo CSV.



**Estructura del archivo CSV generado**

| Columna      | Descripción                                                                           |
|--------------|----------------------------------------------------------------------------------------|
| `patient_id` | Identificador del paciente (nombre del archivo).                                      |
| `sentence`   | La oración completa donde aparece la entidad.                                         |
| `entity`     | El texto exacto extraído &nbsp;—&nbsp;*ej.:* “carcinoma ductal”.                       |
| `label`      | Tipo de entidad &nbsp;—&nbsp;*ej.:* “DISEASE”.                                         |
| `score`      | Confianza del modelo al hacer la predicción (valor entre 0 y 1).  

## **Instalación dependencias y rutas de accesso**

Se instalan las librerias necesarias para ejecutar el scrip. Adicionalmente, se define la ruta donde se encuentran los archivos.



In [ ]:
#Instalar dependencias
!pip install -q transformers torch pandas tqdm huggingface_hub

In [ ]:
#Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Ruta historias (.txt)
DATA_DIR = "/content/drive/MyDrive/MAESTRIA MAIN/2 SMEMESTRE/ANALITICA DE DATOS EN SALUD/TAREA 2/Notas_Cancer_Mama" #Esta en la dirección donde se encuentran las historias clinicas

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **Configuración**
Esta configuración establece los parámetros clave para ejecutar un modelo de extracción de entidades (NER) aplicado a historias clínicas de cáncer de mama. `MODEL_ID` define el modelo especializado que se usará, en este caso uno alojado en Hugging Face y entrenado para textos biomédicos. Como el modelo es privado (gated), se requiere un `HF_TOKEN` válido para autenticar el acceso. `MAX_TOKENS` limita la cantidad de texto que se procesa por bloque (400 tokens) para evitar errores por exceso de longitud. El parámetro `MIN_SCORE` indica el nivel mínimo de confianza (0.80) que debe tener una entidad detectada para ser aceptada. Finalmente, `DEVICE` controla si el modelo se ejecuta en GPU (0) o CPU (-1), siendo la GPU recomendada si está disponible por su mayor velocidad. Esta configuración permite adaptar el script de extracción de entidades a distintos entornos y garantizar resultados confiables.

In [ ]:
#Configuración
import os, re
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from huggingface_hub import login
from tqdm.auto import tqdm
import pandas as pd

MODEL_ID   = "anvorja/breast-cancer-biomedical-ner-sp-1"
HF_TOKEN   = ""  # Pon aqui tu token
MAX_TOKENS = 400
MIN_SCORE  = 0.80
DEVICE     = 0     # 0 para GPU, -1 para CPU

DATA_DIR   = "/content/drive/MyDrive/MAESTRIA MAIN/2 SEMESTRE/ANALITICA DE DATOS EN SALUD/TAREA 2/Notas_Cancer_Mama" #Esta en la dirección donde se encuentran las historias clinicas
OUTPUT_CSV = "/content/drive/MyDrive/MAESTRIA MAIN/2 SEMESTRE/ANALITICA DE DATOS EN SALUD/TAREA 2/entidades.csv" #Esta en la dirección donde salen las historias clinicas procesadas

#Autenticación en Hugging Face
login(token=HF_TOKEN, add_to_git_credential=False)


## **Cargar modelo**

Se carga el **modelo NER** previamente entrenado desde Hugging Face, utilizando el identificador MODEL_ID y un token de autenticación (`HF_TOKEN`). Primero se inicializa el tokenizador, que convierte el texto en una secuencia de tokens comprensibles por el modelo. Luego se carga el modelo de clasificación de entidades, y finalmente se construye un pipeline de tipo ner que integra ambos componentes. Este pipeline permite procesar directamente texto en lenguaje natural para extraer entidades, especificando también si se ejecuta en GPU o CPU mediante el parámetro device.

Además, se definen dos funciones auxiliares que preparan el texto antes de pasarlo al modelo. La función split_sentences() divide el texto completo en oraciones individuales usando signos de puntuación y saltos de línea. Por su parte, chunk_tokens() corta la lista de tokens en fragmentos más pequeños, lo que permite procesar textos largos respetando el límite de entrada del modelo.

In [ ]:
#Cargar modelo y pipeline
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_auth_token=HF_TOKEN)
model     = AutoModelForTokenClassification.from_pretrained(MODEL_ID, use_auth_token=HF_TOKEN)
ner_pipe  = pipeline(
    "ner",
    model=model,
    tokenizer=tokenizer,
    aggregation_strategy="simple",
    device=DEVICE
)

#Funciones auxiliares
def split_sentences(text: str):
    return [s.strip() for s in re.split(r"(?<=[.!?])\s+|\n+", text) if s.strip()]

def chunk_tokens(tokens, size):
    for i in range(0, len(tokens), size):
        yield tokens[i:i+size]


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:934: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:492: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Device set to use cpu


## **Procesamiento**
Se recorre la carpeta de trabajo con una barra de progreso para detectar cada archivo `.txt`. Por cada historia clínica se obtiene un identificador de paciente a partir del nombre del archivo, se lee todo el contenido y se fragmenta en oraciones. Cada oración se divide en tokens y, si supera el límite de entrada del modelo, se trocea en bloques de tamaño controlado. Cada bloque de texto pasa por el pipeline NER, que devuelve un conjunto de entidades con su probabilidad.

Solo se guardan las entidades cuya confianza supera el umbral establecido; de cada una se recoge el identificador del paciente, la oración completa, el texto de la entidad, su etiqueta y la puntuación redondeada. Toda esa información se va acumulando en la lista records, que al final contendrá una estructura lista para convertir a DataFrame y exportar a CSV.

In [ ]:
import time
from tqdm.auto import tqdm

#Tiempo de procesamiento
start = time.perf_counter()

#Procesar todos los .txt y extraer entidades
records = []
for fname in tqdm(sorted(os.listdir(DATA_DIR)), desc="Procesando archivos"):
    if not fname.lower().endswith(".txt"):
        continue
    patient_id = os.path.splitext(fname)[0]
    with open(os.path.join(DATA_DIR, fname), encoding="utf-8") as f:
        text = f.read()
    for sentence in split_sentences(text):
        tokens = sentence.split()
        for toks in chunk_tokens(tokens, MAX_TOKENS):
            for ent in ner_pipe(" ".join(toks)):
                if ent["score"] < MIN_SCORE:
                    continue
                records.append({
                    "patient_id": patient_id,
                    "sentence": sentence,
                    "entity": ent["word"],
                    "label": ent["entity_group"],
                    "score": round(ent["score"], 4)
                })


elapsed = time.perf_counter() - start
print(f"⏱️  Tiempo total: {elapsed:.2f} segundos")

Procesando archivos:   0%|          | 0/106 [00:00<?, ?it/s]

⏱️  Tiempo total: 850.56 segundos


## **Almacenamiento**
Una vez procesadas las historias clinicas, se convierten todos los diccionarios acumulados en `records` a un DataFrame de pandas, lo que permite manejarlos como una tabla estructurada. A continuación, esa tabla se exporta a un archivo CSV en la ruta indicada por `OUTPUT_CSV`, se omite la columna de índice (index=False) y se usa codificación `UTF-8` para mantener correctamente los acentos y caracteres especiales.


In [ ]:
#Guardar en CSV y mostrar primeras filas
df = pd.DataFrame(records)
df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8")
print(f"✅ Guardadas {len(df)} entidades en:")
print(OUTPUT_CSV)
df.head(10)

✅ Guardadas 3407 entidades en:
/content/drive/MyDrive/MAESTRIA MAIN/2 SEMESTRE/ANALITICA DE DATOS EN SALUD/TAREA 2/entidades.csv


,patient_id,sentence,entity,label,score
0,100(1),Motivo de consulta: Paciente no asiste a quimi...,quimioterapia,TREATMENT_NAME,0.9985
1,100(1),Enfermedad actual: Diagnosticos: CA DE MAMA DU...,Diagnosticos,OCURRENCE_EVENT,0.9982
2,100(1),Enfermedad actual: Diagnosticos: CA DE MAMA DU...,CA DE MAMA DUCTAL INFILTRANTE DE MAMA IZQUIERDA,CANCER_CONCEPT,0.9999
3,100(1),Enfermedad actual: Diagnosticos: CA DE MAMA DU...,pT2N2M0,TNM,0.9997
4,100(1),Enfermedad actual: Diagnosticos: CA DE MAMA DU...,Estadio IIIA,STAGE,0.9997
5,100(1),Enfermedad actual: Diagnosticos: CA DE MAMA DU...,RECEPTORES HORMONALES POSITIVOS,BIOMARKER,0.9997
6,100(1),Enfermedad actual: Diagnosticos: CA DE MAMA DU...,HER 2 POSITIVO,BIOMARKER,0.9996
7,100(1),"Mujer con cancer de mama izquierda, estadio IIIA.",cancer de mama izquierda,CANCER_CONCEPT,0.9998
8,100(1),"Mujer con cancer de mama izquierda, estadio IIIA.",estadio IIIA,STAGE,0.9996
9,100(1),"Carcinoma ductual infiltrante, estadio IIIA, H...",Carcinoma ductual infiltrante,CANCER_CONCEPT,0.9996
